<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving forest_zip.zip to forest_zip.zip
Saving mountain_zip.zip to mountain_zip.zip


In [0]:
!unzip mountain_zip.zip

In [0]:
!unzip forest_zip.zip

In [0]:
import os

In [0]:
forest = ['./forest/'+image for image in os.listdir('./forest')]
mountain = ['./mountain/'+image for image in os.listdir('./mountain')]
forest = [ x for x in forest if '.ipynb' not in x ]
forest = [ x for x in forest if '.db' not in x ]
mountain = [ x for x in mountain if ".db" not in x ]
mountain = [ x for x in mountain if ".ipynb" not in x ]

In [0]:
import numpy as np
y0 = np.zeros(len(forest))
y1 =  np.ones(len(mountain))
y = np.hstack((y0,y1))

In [0]:
y = np.reshape(y, (702,1))

In [0]:
from PIL import Image
x = []
for image in (forest + mountain):
  image_array = np.asarray(Image.open(image))
  x.append(image_array)

In [0]:
X = np.array(x) / 255

In [40]:
X.shape, y.shape

((702, 256, 256, 3), (702, 1))

## Instatiate Model

In [0]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

for layer in resnet.layers:
    layer.trainable = False

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model1 = Model(resnet.input, predictions)

In [0]:
model1.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [44]:
stop = EarlyStopping(monitor='acc', min_delta=.001, patience=3)
model1.fit(X, y, epochs=99, callbacks=[stop])

Train on 702 samples
Epoch 1/99
702/702 [==============================] - 4s 5ms/sample - loss: 0.3588 - acc: 0.8761
Epoch 2/99
702/702 [==============================] - 2s 3ms/sample - loss: 0.0300 - acc: 0.9843
Epoch 3/99
702/702 [==============================] - 2s 3ms/sample - loss: 0.0233 - acc: 0.9943
Epoch 4/99
702/702 [==============================] - 2s 3ms/sample - loss: 0.0101 - acc: 0.9972
Epoch 5/99
702/702 [==============================] - 2s 3ms/sample - loss: 0.0397 - acc: 0.9900
Epoch 6/99
702/702 [==============================] - 2s 3ms/sample - loss: 0.0161 - acc: 0.9943
Epoch 7/99
702/702 [==============================] - 2s 3ms/sample - loss: 0.0563 - acc: 0.9758


In [25]:
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential, Model # <- May Use
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(8, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(4, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 125, 125, 32)      18464     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 60, 60, 16)        4624      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 28, 28, 8)       

## Fit Model

In [0]:
model.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [28]:
stop = EarlyStopping(monitor='acc', min_delta=.001, patience=3)
model.fit(x, y, epochs=99, callbacks=[stop])

Train on 702 samples
Epoch 1/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.1550 - acc: 0.9316
Epoch 2/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.1232 - acc: 0.9544
Epoch 3/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.1325 - acc: 0.9516
Epoch 4/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.1301 - acc: 0.9516
Epoch 5/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.1010 - acc: 0.9558
Epoch 6/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.0759 - acc: 0.9715
Epoch 7/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.0571 - acc: 0.9801
Epoch 8/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.5182 - acc: 0.8618
Epoch 9/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.5899 - acc: 0.6880
Epoch 10/99
702/702 [==============================] - 1s 2ms/sample - loss: 0.4913 - acc: 0.8219


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 215, 215, 32)      9632      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 39, 39, 64)        51264     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 9, 9, 64)          102464    
_________________________________________________________________
flatten_1 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)               

In [0]:
# Compile Model


In [0]:
# Fit Model


Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2667 - accuracy: 0.9073 - val_loss: 0.1186 - val_accuracy: 0.9858
Epoch 2/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2046 - accuracy: 0.9073 - val_loss: 0.3342 - val_accuracy: 0.8511
Epoch 3/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1778 - accuracy: 0.9287 - val_loss: 0.2746 - val_accuracy: 0.8723
Epoch 4/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1681 - accuracy: 0.9323 - val_loss: 0.8487 - val_accuracy: 0.5957
Epoch 5/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1606 - accuracy: 0.9394 - val_loss: 0.3903 - val_accuracy: 0.8582


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language